In [1]:
import pandas as pd
import numpy as np
from fastai.tabular.all import *

In [17]:
df = pd.read_pickle("donors.pkl")
sample = pd.read_pickle("sample_eda_done.pkl")

In [18]:
# Variables to use to predict income and wealth1

pred_income_x_cont = [
    "AGE",
    "IC1",
    "IC2",
    "IC3",
    "IC4",
    "IC5",
    "IC6",
    "IC7",
    "IC8",
    "IC9",
    "IC10",
    "IC11",
    "IC12",
    "IC13",
    "IC14",
    "RAMNTALL",
]

pred_income_x_cat = [
    "TCODE",
    "STATE",
    "ZIP",
    "MAJOR",
    "MDMAUD",
    "DOMAIN",
    "HOMEOWNR",
    "NUMCHLD",
    "ageGroup",
    "GENDER",
    "WEALTH2",
    "MSA",
    "ADI",
    "DMA",
    
]

In [21]:
cont_names, cat_names = cont_cat_split(sample, max_card=40)
# df_pred = df[pred_income_x_cat + pred_income_x_cont + ["cat_INCOME"]].copy()
df_pred = df[cat_names + cont_names].copy()
train_df = df_pred[~df_pred["INCOME"].isnull()]
test_df = df_pred[df_pred["INCOME"].isnull()].drop(["INCOME"], axis=1)

In [23]:
train_df.INCOME.unique()

array([6., 3., 1., 4., 2., 7., 5.])

In [27]:
splits = RandomSplitter(valid_pct=0.3)(range_of(train_df))

to = TabularPandas(train_df, procs=[Categorify, FillMissing, Normalize],
                  cat_names = cat_names,
                  cont_names = cont_names,
                  y_names = "INCOME",
                  splits=splits)

dls = to.dataloaders(bs=64)

learn = tabular_learner(dls, metrics=accuracy)

In [28]:
learn.fit_one_cycle(1, 0.03)

epoch,train_loss,valid_loss,accuracy,time
0,0.023450,221.476105,0.000000,01:11


In [16]:
dl = learn.dls.test_dl(test_df)
preds = learn.get_preds(dl=dl, with_decoded=True, with_input=True)

In [22]:
set(preds[2].numpy())

{0, 1, 2}

In [63]:
preds[2].size()

torch.Size([21286])